In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

import faiss

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

from catboost import Pool, CatBoostClassifier

In [2]:
DATA_PATH = "C:\\Users\\Natalia\\Desktop\\Masterskaya\\"

In [3]:
df_base = pd.read_csv(f"{DATA_PATH}base.csv", index_col=0)
df_train = pd.read_csv(f"{DATA_PATH}train.csv", index_col=0)
df_validation = pd.read_csv(f"{DATA_PATH}validation.csv", index_col=0)
df_validation_answer = pd.read_csv(f"{DATA_PATH}validation_answer.csv", index_col=0)

In [4]:
targets = df_train["Target"]
df_train.drop("Target", axis=1, inplace=True)

In [5]:
df_base.drop(["21", "25", "33", "44", "59", "65", "70"], axis=1, inplace=True)
df_train.drop(["21", "25", "33", "44", "59", "65", "70"], axis=1, inplace=True)
df_validation.drop(["21", "25", "33", "44", "59", "65", "70"], axis=1, inplace=True)

In [6]:
# Нормализация данных
def normalize_with_indices(df):
    original_indices = df.index
    
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    
    df_normalized = pd.DataFrame(df_scaled, index=original_indices, columns=df.columns)
    return df_normalized

In [7]:
df_base_scaled = normalize_with_indices(df_base)
df_train_scaled  = normalize_with_indices(df_train)
df_validation_scaled = normalize_with_indices(df_validation)

In [8]:
dims = df_base_scaled.shape[1]
n_cells = 10

quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

In [9]:
idx_l2.train(np.ascontiguousarray(df_base_scaled).astype('float32'))

idx_l2.add(np.ascontiguousarray(df_base_scaled).astype('float32'))

In [10]:
base_index = {k: v for k, v in enumerate(df_base_scaled.index.to_list())}

In [11]:
%%time
k_neighbours = 100
vecs, idx = idx_l2.search(np.ascontiguousarray(df_train_scaled.values).astype('float32'), k_neighbours)

Wall time: 8min 18s
Parser   : 211 ms


In [12]:
acc = 0
for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(idx))

67.132
